# Chapter 6 데이터 로딩과 저장, 파일 형식

- 입출력의 범주
    - 텍스트 파일을 이용하는 법
    - 데이터베이스를 이용하는 법
    - 웹 API를 이용해서 네트워크를 통해 불러오는 방법

## 6.1 텍스트 파일에서 데이터를 읽고 쓰는 법

#### [Table] pandas 파일 파싱 함수

<details>
<summary>pandas 파일 파싱 함수</summary>
<div markdown="1">

|함수|설명|
|:--|:--|
|read_csv|파일, URL 또는 파일과 유사한 객체로부터 구분된 데이터를 읽어온다. 데이터 구분자는 쉼표(,)를 기본으로 한다.|
|read_table|파일, URL 또는 파일과 유사한 객체로부터 구분된 데이터를 읽어온다. 데이터 구분자는 탭('\t')을 기본으로 한다.|
|read_fwf|고정폭 컬럼 형식에서 데이터를 읽어온다 (구분자가 없는 데이터).|
|read_clipboard|클립보드에 있는 데이터를 읽어오는 read_table 함수. 웹페이지에서 표를 읽어올 때 유용하다.|
|read_excel|엑셀 파일(XLS, XLSX)에서 표 형식의 데이터를 읽어온다.|
|read_hdf|pandas에서 저장한 HDFS 파일에서 데이터를 읽어온다.|
|read_html|HTML 문서 내의 모든 테이블의 데이터를 읽어온다.|
|read_json|JSON 문자열에서 데이터를 읽어온다.|
|read_msgpack|메시지팩 바이너리 포맷으로 인코딩된 pandas 데이터를 읽어온다.|
|read_pickle|파이썬 피클 포맷으로 저장된 객체를 읽어온다.|
|read_sas|SAS 시스템의 사용자 정의 저장 포맷으로 저장된 데이터를 읽어온다.|
|read_sql|SQL 쿼리 결과를 pandas의 DataFrame 형식으로 읽어온다.|
|read_stata|Stata 파일에서 데이터를 읽어온다.|
|read_feather|Feather 바이너리 파일 포맷으로부터 데이터를 읽어온다.|
    
</div>
</details>

- 파일을 읽어오는 함수들의 옵션
    - 색인 
        - 반환하는 DataFrame에서 하나 이상의 컬럼을 색인으로 지정할 수 있다. 파일이나 사용자로부터 컬럼 이름을 받거나 아무것도 받지 않을 수 있다.
    - 자료형 추론과 데이터 변환
        - 사용자 정의 값 변환과 비어 있는 값을 위한 사용자 리스트를 포함한다.
    - 날짜 분석
        - 여러 컬럼에 걸쳐 있는 날짜와 시간 정보를 하나의 컬럼에 조합해서 결과에 반영한다.
    - 반복
        - 여러 개의 파일에 걸쳐 있는 자료를 반복적으로 읽어올 수 있다.
    - 정제되지 않은 데이터 처리
        - 로우나 꼬리말, 주석 건너뛰기 또는 천 단위마다 쉼포료 구분된 숫자 같은 사소한 것들의 처리를 해준다.

> 사전에 파일의 내용을 확인하는 방법

In [3]:
!type ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


> ex1.csv 파일은 쉼표로 구분되어 있기 때문에 `read_csv`파일로 읽어올 수 있다

In [125]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> read_table에 구분자를 쉼표로 지정해서 읽어올 수도 있다.

In [8]:
df = pd.read_table('ex1.csv', sep = ',')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> 파일에 컬럼 이름이 없는 경우 pandas가 자동으로 컬럼 이름을 생성하도록 하거나, 직접 컬럼 이름을 지정할 수 있다.

In [11]:
!type ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [9]:
pd.read_csv('ex2.csv', header = None)    # pandas가 자동으로 컬럼 이름을 생성하도록 하는 경우.

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
pd.read_csv('ex2.csv', names = ['a','b','c','d','message'])    # 사용자가 컬럼 이름을 정해주는 경우

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> 특정 컬럼을 지정하여 데이터 프레임의 색인(index)으로 지정할 수 있다.

In [13]:
names = ['a','b','c','d','message']

pd.read_csv('ex2.csv', names = names, index_col = 'message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


> 계층적 색인을 지정하고 싶다면 컬럼 번호나 이름의 리스트를 넘기면 된다.

In [15]:
!type csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [18]:
parsed = pd.read_csv('csv_mindex.csv',
                     index_col = ['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

> 고정된 구분자 없이 공백이나 다른 패턴으로 필드를 구분해놓은 경우, 정규 표현식으로도 처리할 수 있다.

In [24]:
list(open('ex3.txt'))   # 해당 파일은 여러 개의 공백 문자로 구분되어 있음

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [28]:
result = pd.read_table('ex3.txt', sep = '\s+')    # 공백을 의미하는 정규 표현식 \s+을 구분자로 삼을 수 있다.
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


> skiprows를 이용해 원하는 로우를 건너뛰고 데이터를 불러올 수 있다.

In [29]:
!type ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [30]:
pd.read_csv('ex4.csv', skiprows = [0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> pandas는 NA나 NULL처럼 흔히 통용되는 문자를 비어 있는 값으로 사용한다.

In [31]:
!type ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [32]:
result = pd.read_csv('ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


> na_values 옵션은 리스트나 문자열 집합을 받아서 누락된 값을 처리한다.  
> 컬럼마다 다른 NA 문자를 사전값으로 넘겨서 처리할 수도 있다.

In [34]:
!type ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [35]:
sentinels = {'message' : ['foo','NA'], 
             'something' : ['two']}

pd.read_csv('ex5.csv', na_values = sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


#### [Table] 파서 함수에서 자주 사용되는 인자

<details>
<summary>read_csv와 read_table 함수 인자</summary>
<div markdown="1">

|인자|설명|
|:--|:--|
|path|파일시스템에서의 위치, URL, 파일 객체를 나타내는 문자열|
|sep 또는 delimiter|필드를 구분하기 위해 사용할 연속된 문자나 정규 표현식|
|header|컬럼 이름으로 사용할 로우 번호. 기본값은 0(첫번째 로우)이며 헤더가 없을 경우에는 None으로 지정할 수 있다.|
|index_col|색인으로 사용할 컬럼 번호나 이름, 계층적 색인을 지정할 경우 리스트를 넘길 수 있다.|
|names|컬럼 이름으로 사용할 리스트, header = None과 함께 사용한다.|
|skiprows|파일의 시작부터 무시할 행 수 또는 무시할 로우 번호가 담긴 리스트|
|na_values|NA 값으로 처리할 값들의 목록|
|comment|주석으로 분류되어 파싱하지 않을 문자 혹은 문자열|
|parse_dates|날짜를 datetime으로 변환할지 여부. 기본값은 False이며, True일 경우 모든 컬럼에 적용된다. 컬럼의 번호나 일므을 포함한 리스트를 넘겨서 변환할 컬럼을 지정할 수 잇는데, [1,2,3]을 넘기면 각각의 컬럼을 datetime으로 변환하며, [[1,3]]을 넘기면 1,3번 컬럼을 `조합`해서 하나의 datetime으로 변환한다.|
|keep_date_col|여러 컬럼을 datetime으로 변환했을 경우 원래 컬럼을 남겨둘지 여부. 기본값은 True|
|converters|변환 시 컬럼에 적용할 함수를 지정한다. 예를 들어 {'foo':f}는 'foo' 컬럼에 f 함수를 적용시킨다. 전달하는 사전의 키값은 컬럼 이름이나 번호가 될 수 있다.|
|dayfirst|모호한 날짜 형식일 경우 국제 형식으로 간주한다(7/6/2012는 2012년 6월 7일로 간주한다). 기본값은 False|
|date_parser|날짜 변환 시 사용할 함수|
|nrows|파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정|
|iterator|파일을 조금씩 읽을 때 사용하도록 TextParser 객체를 반환하도록 한다. 기본값은 False|
|chunksize|TextParser 객체에서 사용할 한 번에 읽을 파일의 크기|
|skip_footer|파일의 끝에서 무시할 라인 수|
|verbose|파싱 결과에 대한 정보를 출력한다. 숫자가 아닌 값이 들어 있는 컬럼에 누락된 값이 있다면 줄 번호를 출력해준다. 기본값은 False|
|encoding|유니코드 인코딩 종류를 지정한다. UTF-8로 인코딩된 텍스트일 경우 'utf-8'로 지정한다.|
|squeeze|만일 컬럼이 하나뿐이라면 Series 객체를 반환한다. 기본값은 False|
|thousands|숫자를 천 단위로 끊을 때 사용할 ','나 '.' 같은 구분자|
    
</div>
</details>

### 6.1.1 텍스트 파일 조금씩 읽어오기

- 매우 큰 파일을 처리할 때 인자를 제대로 주었는지 알아보기 위해 파일의 일부분만 읽어보거나 여러 파일 중에서 몇 개의 파일만 읽어서 확인하고 싶은 경우 사용

In [47]:
# pandas의 출력 설정; 최대 10개의 데이터만 출력됨
pd.options.display.max_rows = 10

In [48]:
pd.read_csv('examples/ex6.csv')

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


> 파일 전체를 읽는 대신 처음 몇 줄만 읽어보고 싶다면 `nrows` 옵션을 주면 된다.

In [49]:
pd.read_csv('examples/ex6.csv', nrows = 5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


> 파일을 여러 조각으로 나누어서 읽고 싶다면 `chunksize`옵션으로 로우의 개수를 주면 된다.  
> read_csv에서 반환된 TextParser 객체를 이용해서 chunksize에 따라 분리된 파일들을 순회할 수 있다.  
> 예를 들어 ex6.csv 파일을 순회하면서 'key' 컬럼에 있는 값을 세어보려면 다음처럼 하면 된다.

In [93]:
chunker = pd.read_csv('examples/ex6.csv', chunksize = 1000)
chunker

In [94]:
tot = pd.Series([])

for piece in chunker :
    tot = tot.add(piece['key'].value_counts(), fill_value = 0)
    
tot = tot.sort_values(ascending=False)
tot[:10]

<ipython-input-94-680a360461e9>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### 6.1.2 데이터를 텍스트 형식으로 기록하기

In [106]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


> DataFrame의 to_csv 메서드를 이용하면 데이터를 쉼표로 구분된 형식으로 파일에 쓸 수 있다 (콘솔에서 확인할 수 있도록 실제 파일로 기록하지 않고 sys.stdout에 결과를 기록하도록 했다).

In [118]:
import sys 

data.to_csv(sys.stdout)

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


> 다른 구분자도 사용 가능하다 

In [119]:
data.to_csv(sys.stdout, sep = '|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


> 결과에서 누락된 값이 비어 있는 문자열로 나타나는데, 이것 역시 원하는 값으로 지정 가능하다.

In [120]:
data.to_csv(sys.stdout, na_rep = "NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


> 다른 옵션을 명시하지 않으면 로우와 컬럼 이름이 기록된다.  
> 로우와 컬럼 이름을 포함하지 않으려면 다음과 같이 한다.

In [121]:
data.to_csv(sys.stdout, index = False, header = False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


> 컬럼의 일부분만 기록할 수도 있으며, 순서를 직접 지정할 수도 있다.

In [122]:
data.to_csv(sys.stdout, index = False, columns = ['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


> Series에도 to_csv 메서드가 존재한다.

In [127]:
dates = pd.date_range('1/1/2000', periods = 7)
ts = pd.Series(np.arange(7), index = dates)

ts.to_csv(sys.stdout)

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### 6.1.3 구분자 형식 다루기 

### 6.1.4 JSON 데이터

- $\text{JSON}^{\text{JavaScript Object Notation}}$은 웹브라우저와 다른 애플리케이션이 HTTP 요청으로 데이터를 보낼 때 널리 사용하는 표준 파일 형식 중 하나.  
- JSON은 CSV 같은 표 형식의 텍스트보다 좀 더 유연한 데이터 형식.

> JSON 데이터를 읽고 쓸 수 있는 파이썬 라이브러리는 몇 가지가 있으나 여기서는 파이썬 표준 라이브러리인 json을 사용함.  
> JSON 문자열을 파이썬 형태로 변환하기 위해서는 json.loads를 사용.

In [135]:
import json

obj = '''
{"name" : "Wes",
 "places_lived" : ["United States", "Spain", "Germany"],
 "pet" : null,
 "siblings" : [{"name" : "Scott", "age" : 30, "pets" : ["Zeus","Zuko"]},
               {"name" : "Katie", "age" : 38, "pets" : ["Sixes","Stache","Cisco"]}]
}
'''

In [137]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

> json.dumps는 파이썬 객체를 JSON 형태로 변환한다.

In [138]:
asjson = json.dumps(result)

> JSON 객체의 리스트를 사전을 담고 있는 리스트로 변환하여 DataFrame 생성자로 넘기고 데이터 필드를 선택할 수 있다.

In [140]:
siblings = pd.DataFrame(result['siblings'], columns = ['name','age','pets'])
siblings

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


> pandas.read_json은 자동으로 JSON 데이터셋을 Series나 DataFrame으로 변환할 수 있다.  
> 별다른 옵션이 주어지지 않았을 경우 pandas.read_json은 JSON 배열에 담긴 각 객체를 테이블의 로우로 간주한다.

In [143]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


> pandas의 데이터를 JSON으로 저장하는 한 가지 방법은 Series나 DataFrame의 to_json 함수를 이용하는 것.

In [144]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [145]:
print(data.to_json(orient = 'records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### 6.1.5 XML과 HTML : 웹 스크레핑

- 파이썬에는 lxml, Beautiful Soup, 그리고 html5lib 같은 HTML과 XML 형식의 데이터를 읽고 쓸 수 있는 라이브러리가 많다.
- 기 중 lxml은 가장 빠르게 동작하고 깨진 HTML과 XML 파일도 잘 처리해준다.
- pandas에 있는 read_html 내장함수는 lxml이나 Beautiful Soup 같은 라이브러리를 사용해서 자동으로 HTML을 파싱, DataFrame으로 변환해준다.

> 사용법을 알아보기 위해 미연방예금보험공사(FDIC)에서 부도은행을 보여주는 HTML을 다운로드 하자.  
(https://www.fdic.gov/bank/individual/failed/banklist.html)

In [147]:
# pip install lxml beautifulsoup4 html5lib

> pandas.read_html 함수는 기본적으로 \<table> 태그 안에 있는 모든 표 형식의 데이터 파싱을 시도한다.  
> 결과는 DataFrame 객체의 리스트에 저장된다.

In [151]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)

1

In [152]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


> 이후 데이터 정제와 연도별 부도은행 수 계산 등의 분석을 시작할 수 있다.

In [160]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts().sort_index()

2000     2
2001     4
2002    11
2003     3
2004     4
        ..
2012    51
2013    24
2014    18
2015     8
2016     5
Name: Closing Date, Length: 15, dtype: int64

## 6.2 이진 데이터 형식

- 데이터를 효율적으로 저장하는 가장 손쉬운 방법은 파이썬에 기본으로 내장되어 있는 `pickle 직렬화`를 하용해 데이터를 이진 형식으로 저장하는 것.
- pandas 객체는 모두 pickle을 이용해서 데이터를 저장하는 to_pickle 메서드를 가지고 있다.

In [162]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [163]:
frame.to_pickle('examples/frame_pickle')

> pickle로 직렬화된 객체는 내장 함수인 pickle로 직접 불러올 수 있음.  
> 혹은 좀 더 편리한 pandas.read_pickle 메서드를 이용하여 불러올 수 있음.

In [164]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 6.2.1 HDF5 형식 사용하기 (tables packages 문제 있음 나중에)

- HDF5는 대량의 과학 계산용 배열 데이터를 저장하기 위해 고안된 파일 포맷임.
- HDF는 Hierarchical Data Format의 약자로 계층적 데이터 형식이라는 뜻.
- 각각의 HDF5 파일은 여러 개의 데이터 셋을 저장하고 부가 정보를 기록할 수 있음. 
- HDF5는 다양한 압축 기술을 사용해서 $온더플라이^{\text{on-the-fly}}$(실시간) 압축을 지원하며 반복되는 패턴을 가진 데이터를 좀 더 효과적으로 저장할 수 있다. 
- 메모리에 모두 적재할 수 없는 엄청나게 큰 데이터를 아주 큰 배열에서 필요한 작은 부분들만 효과적으로 읽고 쓸 수 있는 훌륭한 선택이다.

> PyTables나 h5py 라이브러리를 이용하여 직접 HDF5 파일에 접근하는 것도 가능.  
> pandas는 Series나 DataFrame 객체로 간단히 저장할 수 있는 고수준의 인터페이스를 제공함.  
> HDFStore 클래스는 사전처럼 작동하며 세밀한 요구 사항도 잘 처리해 줌.

In [16]:
frame = pd.DataFrame({'a' : np.random.randn(100)})

In [23]:
store = pd.HDFStore('mydata.h5')

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

### 6.2.2 마이크로소프트 엑셀 파일에서 데이터 읽어오기 

- pandas는 ExcelFile 클래스나 pandas.read_excel 함수를 사용해서 마이크로소프트 엑셀 2003 이후 버전의 데이터를 읽어올 수 있다.
- 내부적으로 이들 도구는 XLS 파일과 XLSX 파일을 읽기 위해 각각 xlrd와 openpyxl 패키지를 사용하므로 미리 설치가 필요하다.

In [25]:
# pip install xlrd openpyxl

> ExcelFile 클래스를 사용하려면 xls나 xlsx파일의 경로를 지정하여 `객체를 생성` 해야한다.

In [26]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

> 시트에 있는 데이터는 parse 함수를 이용해서 DataFrame으로 읽어올 수 있다.

In [29]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


> 한 파일에서 `여러 시트`를 읽어오려면 ExcelFile 객체를 생성하면 빠르다.  
> `하나의 시트`만 필요한 경우 간단하게 pandas.read_excel에 파일 이름만 넘겨도 된다.

In [31]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


> pandas 데이터를 엑셀 파일로 저장하고 싶다면 ExcelWrite를 생성해서 데이터를 기록하고 pandas 객체의 to_excel 메서드로 넘기면 된다.

In [32]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

frame.to_excel(writer, 'Sheet1')

writer.save()

> ExcelWriter를 사용하지 않고 to_excel 메서드에 파일 경로만 넘겨도 된다.

In [33]:
frame.to_excel('examples/ex2.xlsx')

## 6.3 웹 API와 함께 사용하기

- 파이썬으로 API를 사용하는 법 중 가장 손쉬운 방법은 requests 패키지를 이용하는 것.

> pandas 깃허브에서 최근 30개의 이슈를 가져오려면 requests 라이브러리를 이용해서 다음과 같은 GET HTTP 요청을 생성하면 된다.

In [34]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

resp = requests.get(url)
resp

<Response [200]>

> 응답 객체의 json 메서드는 JSON의 내용을 파이썬 사전 형태로 변환한 객체를 반환한다.

In [38]:
data = resp.json()
data[0]['title']

'TST: df.combine_first duplicates rows for nan index values (#39881)'

> 해당 data를 바로 DataFrame으로 생성하고 관심이 있는 필드만 따로 추출할 수 있다.

In [41]:
issues = pd.DataFrame(data, columns = ['number','title','labels','state'])
issues

,number,title,labels,state
0,43066,TST: df.combine_first duplicates rows for nan ...,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
1,43065,REF: Move BaseIndexers and indexer.py function...,"[{'id': 127681, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
2,43064,BUG: Allow named aggregation with Resampler.agg,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,43063,BENCH: fix misleading json asvs,"[{'id': 732775912, 'node_id': 'MDU6TGFiZWw3MzI...",open
4,43061,DOC: Updating grammar for clarity and consistency,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
5,43060,BUG: nlargest/nsmallest can now consider nan v...,"[{'id': 47223669, 'node_id': 'MDU6TGFiZWw0NzIy...",open
6,43059,PERF: read_stata,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
7,43058,fixed transform docs,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
8,43056,ENH:included anti join functionality,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
9,43054,REGR: SeriesGrouper using Timestamp index,"[{'id': 211840, 'node_id': 'MDU6TGFiZWwyMTE4ND...",open


## 6.4 데이터베이스와 함께 사용하기

- 비즈니스 관점에서 대부분의 데이터는 텍스트 파일이나 엑셀 파일로 저장하지 않고 SQL 기반의 관계형 데이터베이스(SQL 서버, PostgreSQL, MySQL)를 많이 사용함.
- SQL에서 데이터를 읽어와서 DataFrame에 저장하는 것은 꽤 직관적이며 pandas에는 이 과정을 간결하제 해주는 몇 가지 함수가 있음.
- 파이썬 내장 sqlite3 드라이버를 사용해서 SQLite 데이터베이스를 이용할 수 있음.

In [42]:
import sqlite3

In [48]:
query = '''
CREATE TABLE test
(a VARCHAR(20),
b VARCHAR(20),
c REAL,
d INTEBER);'''

In [49]:
con = sqlite3.connect('mydata.sqlite')

In [50]:
con.execute(query)

In [51]:
con.commit()

> 데이터의 입력

In [53]:
data = [('Atlanta','Georgia',1.25,6),
        ('Tallahassee','Florida',2.6,3),
        ('Sacramento','California',1.7,5)]

stmt = "INSERT INTO test VALUES(?,?,?,?)"

con.executemany(stmt, data)

In [55]:
con.commit()

> 대부분의 파이썬 SQL 드라이버(PyODBC, psycopg2, MySQLdb, pymsqql 등)는 테이블에 대해 select 쿼리를 수행하면 튜플 리스트를 반환한다.

In [59]:
cursor = con.execute('select * from test')

rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

> 

> 반환된 튜플 리스트를 DataFrame 생성자에 바로 전달해도 되지만, 컬럼 이름을 지정해주면 더 편함.  
> cursor의 description 속성을 활용하자.

In [60]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [61]:
pd.DataFrame(rows, columns = [x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


> 유명한 파이썬 SQL 툴킷인 SQLAlchemy (SQL알케미) 프로젝트는 SQL 데이터베이스 간의 일반적인 차이점을 추상화하여 제공함.  
> pandas는 read_sql 함수를 제공하여 SQLAlchemy의 일반적인 연결을 이용해 쉽게 데이터를 읽을 수 있게 함.  
> 다음은 SQLAlchemy를 사용하여 같은 SQLite 데이터베이스에 접속하고 앞서 만든 테이블에서 데이터를 읽어오는 예다.

In [64]:
import sqlalchemy as sqla

In [65]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
